# Adding FHIR data to a FHIR server

Your project may involve ingesting data into FHIR format and sending it to a FHIR server. This notebook is a quick example of how this can be achieved with IRIS health. 

This tutorial will assume you have a FHIR server set up with IRIS-health as described in the [FHIR Server quickstart tutorial](../Tutorial/0-FHIR-server-setup.md). If you have set up the FHIR server by different methods, the differences will be where the requests are send (Endpoint URL, username and password) but the content will be the same. 


## Add existing FHIR files to the FHIR server

You can also add a bundle of FHIR resources to a FHIR Server using HTTP requests - the FHIR server will act the resources to the correct place. For an example, I am going to add 5 patient bundles - that is the complete medical history of the patient. These have been synthetically generated by Synthea. The instructions for this are a [separate tutorial](Creating_synthetic_FHIR-data.md), but are super simple - use the command: 

    docker run --rm -v $PWD/output:/output --name synthea-docker intersystemsdc/irisdemo-base-synthea:version-1.3.4 -p 5

Where -p denotes the number of patients to generate.

First, lets look at the files: 

In [ ]:
import os
os.listdir("./output/fhir")

['hospitalInformation1760013140357.json',
 'Jarred626_Walsh511_ea6f7f29-8553-4456-8fa9-3d629ffcd5f7.json',
 'Krista314_Mosciski958_79fdc511-4556-435d-b922-7a55d0d6e409.json',
 'Kristofer887_Schowalter414_a7b87618-d13a-4f02-8974-010aa35fb759.json',
 'Marshall526_Mosciski958_dbc8773a-10ac-42e5-9c8c-c8d269ed9179.json',
 'Milissa240_Gaylord332_db7fc62f-0f83-484f-bf84-40e85fc26c4d.json',
 'practitionerInformation1760013140357.json']

In [ ]:
import json
import requests
from requests.auth import HTTPBasicAuth

server_url = "http://localhost:32783/csp/healthshare/demo/fhir/r4/"
username = "_SYSTEM"
password = "ISCDEMO"

#Iterate over the files
for file in os.listdir("./output/fhir"):
    # Read the File
    with open("./output/fhir/"+file, "r")as f:
        json_data = json.load(f)
        
    res = requests.post( server_url,
                         json=json_data,
                         headers={"Content-Type": "application/fhir+json"}, 
                         auth=HTTPBasicAuth(username, password))
    print(res)
    if res.status_code!=200:
        print(res.json())

Lets just check the patient resources were added: 

In [ ]:
res = requests.get(baseURL+endpoint, headers=get_headers, auth=HTTPBasicAuth(username, password))
print(res)
#print(res.json())
data = res.json() 

for entry in data.get("entry", [])[-5:]: ## Get the last 5 entries
    patient = entry.get('resource', {}) ## Get the patient resource
    print(patient.get('id'), patient.get('name'), patient.get('gender')) ## Print the patient info


<Response [200]>
3615 [{'use': 'official', 'family': 'Walsh511', 'given': ['Jarred626']}] male
3878 [{'use': 'official', 'family': 'Mosciski958', 'given': ['Krista314'], 'prefix': ['Mrs.']}, {'use': 'maiden', 'family': 'Frami345', 'given': ['Krista314'], 'prefix': ['Mrs.']}] female
4363 [{'use': 'official', 'family': 'Schowalter414', 'given': ['Kristofer887'], 'prefix': ['Mr.']}] male
4510 [{'use': 'official', 'family': 'Mosciski958', 'given': ['Marshall526'], 'prefix': ['Mr.']}] male
5072 [{'use': 'official', 'family': 'Gaylord332', 'given': ['Milissa240'], 'prefix': ['Ms.']}] female


And there we have it! How to add FHIR resources into a FHIR server running with InterSystems IRIS for Health. I hope you have found this useful! 


## Adding New FHIR data to a FHIR Server

I am going to add FHIR data to the server in two steps, firstly I am going to create a valid fhir resource - the python package [`fhir.resources`](https://github.com/nazrulworld/fhir.resources) is going to help with this. Then I will send a POST request to the server using the standard http `requests` library. 

### Step 1: Create valid resources

First of all, install the dependancy - this is [fhir.resources](https://github.com/nazrulworld/fhir.resources). There is much more information about its usage on the linked repo than I am going to include here, so check it out if you want more information


In [ ]:
pip install fhir.resources 

You then need to import the type of resource you would like to make. I am going to start by creating a new patient using a JSON format object:

In [25]:
from fhir.resources.patient import Patient

In [115]:
data = { ## Create a data object that resembles a fhir resource
    "name": [{
        "use": "official",
        "family": "Kent", 
        "given": ["Clark"]  ## As you can have multiple given names (middle names) this needs to be a list item
    }],
    "birthDate":"1965-02-12", "gender":"male"}
superman = Patient.model_validate(data)

In [103]:
print(superman.birthDate)
print(superman.name)

1965-02-12
[HumanName(fhir_comments=None, extension=None, id=None, family='Kent', family__ext=None, given=['Clark'], given__ext=None, period=None, prefix=None, prefix__ext=None, suffix=None, suffix__ext=None, text=None, text__ext=None, use='official', use__ext=None)]


Alternatively we can use a more Pythonic type way:

In [116]:
batman = Patient.model_construct()


batman.name = [{}] # The name part of the patient resource is a bit of a pain! 
batman.id = "500"
batman.name[0].family = "Wayne"
batman.name[0].given = ["Bruce"]
batman.birthDate = "1982-01-15"
batman.gender= "male"

#### Accesing model data

In [105]:
batman_model = batman.model_dump_json() 
print(batman_model,'\n' ,  type(batman_model), '\n\n') ## This returns a JSON string

superman_model = superman.model_dump() ## This returns a json object (or python dict)

print(superman_model, '\n', type(superman_model))

{"resourceType":"Patient","id":"500","name":[{"family":"Wayne","given":["Bruce"]}],"gender":"Male","birthDate":"1982-01-15"} 
 <class 'str'> 


{'resourceType': 'Patient', 'name': [{'use': 'official', 'family': 'Kent', 'given': ['Clark']}], 'gender': 'Male', 'birthDate': datetime.date(1965, 2, 12)} 
 <class 'dict'>


There are classes for every fhir resource defined in the FHIR specification, so a similar method can be used with any resource you may need - Here I'm going to create a DocumentReference resource. If you have worked through the main demo in this repository, you may be familiar with this resource type, but it can be used to link any different type of documents. Here I am going to load a premade clinical note into a document reference: 


In [164]:
from fhir.resources.documentreference import DocumentReference
from datetime import datetime
note_resource = DocumentReference.model_construct()


with open("note_data/clinical_note.txt", "r", encoding="utf-8") as f:
    note = f.read()
    

note_resource.status = "current"
note_resource.date = datetime(2025,9,10)
note_resource.subject = {"reference": "500"}
note_resource.content = [{ "attachment":{"contentType": "text/plain; charset=utf-8", "data": note.encode("utf-8").hex()}}]


### Step 2: Adding data to the server

Similar to how we accessed the data with http requests in the [Accessing-FHIR-resources](./Accessing-FHIR-resources.ipynb) demo, we can add the FHIR resources we have created to the server using HTTP requests. We do need to add basic authorisation. 

In [120]:
import requests
from requests.auth import HTTPBasicAuth

## Credentials
username = "_System"
password = "ISCDEMO"

## FHIR server location
baseURL = "http://localhost:32783/csp/healthshare/demo/fhir/r4/"
headers ={    "Content-Type": "application/fhir+json"}
endpoint = "Patient"


res = requests.post(baseURL+endpoint, 
                    data=superman.model_dump_json(), ## Give a JSON string as data - giving it a Python dict may cause errors because of the datetime object.
                    headers= headers, ## We are defining the content type (FHIR+JSON) in the header
                    auth=HTTPBasicAuth(username, password))
print(res)

<Response [201]>


In [ ]:
## If you have any error message, you can find some more information with 
print(res.json())
## This will error if the data has been added successfully though!

Response 201 means it has been successfully added!

In [117]:
res = requests.post(baseURL+endpoint, data= batman.model_dump_json(), headers= headers, auth=HTTPBasicAuth(username, password))

print(res)

<Response [201]>


#### Checking data is added
Lets just query the endpoint to see whether we have added them correctly:

In [148]:
## FHIR server location
baseURL = "http://localhost:32783/csp/healthshare/demo/fhir/r4/"

get_headers ={    "Accept": "application/fhir+json"}

endpoint = "Patient"


res = requests.get(baseURL+endpoint, headers=get_headers, auth=HTTPBasicAuth(username, password))
print(res)
#print(res.json())
data = res.json() 

for entry in data.get("entry", [])[5:]: ## Get the last 5 entries
    patient = entry.get('resource', {}) ## Get the patient resource
    print(patient.get('id'), patient.get('name'), patient.get('gender')) ## Print the patient info


<Response [200]>
32 [{'use': 'official', 'family': 'Lemke', 'given': ['Tish'], 'prefix': ['Mrs.']}, {'use': 'maiden', 'family': 'Johnston', 'given': ['Tish'], 'prefix': ['Mrs.']}] female
3596 [{'use': 'official', 'family': 'Kent', 'given': ['Clark']}] None
3597 [{'use': 'official', 'family': 'Kent', 'given': ['Clark']}] None
3598 [{'family': 'Wayne', 'given': ['Bruce']}] None
3599 [{'family': 'Wayne', 'given': ['Bruce']}] male
3600 [{'use': 'official', 'family': 'Kent', 'given': ['Clark']}] male


So we have added them! Because we have used a POST request, they are automatically assigned an ID number. We could use a put request if we want to keep the ID number we've assigned them.


### Adding Document Reference

Lets do the same with the document reference resource - only thing we need to change is the endpoint and the data:

In [131]:
res = requests.post(baseURL+"DocumentReference", data=note_resource.model_dump_json(),
                   headers={"Content-Type": "application/fhir+json"}, auth=HTTPBasicAuth(username, password))

print(res)
print(res.json())

<Response [400]>
{'resourceType': 'OperationOutcome', 'issue': [{'severity': 'error', 'code': 'invalid', 'diagnostics': '<HSFHIRErr>MalformedRelativeReference', 'details': {'text': "The reference value '500' in property (subject) of Type 'DocumentReference' is malformed"}, 'expression': ['DocumentReference.subject']}, {'severity': 'error', 'code': 'invalid', 'diagnostics': '<HSFHIRErr>MalformedValue', 'details': {'text': "The value '2025-09-10T00:00:00' of Property 'date' of Type 'DocumentReference' is a malformed 'instant'. It should match the Regex '([0-9]([0-9]([0-9][1-9]|[1-9]0)|[1-9]00)|[1-9]000)-(0[1-9]|1[0-2])-(0[1-9]|[1-2][0-9]|3[0-1])T([01][0-9]|2[0-3]):[0-5][0-9]:([0-5][0-9]|60)(\\.[0-9]+)?(Z|(\\+|-)((0[0-9]|1[0-3]):[0-5][0-9]|14:00))'"}, 'expression': ['DocumentReference.date']}]}


Ok, so the fhir.resources doesnt completely validate our data - the FHIR server will reject bad data! Lets work through these issues: 

In [139]:
for issue in res.json()["issue"]:
    print(issue["details"]["text"])
    print('\n')

The reference value '500' in property (subject) of Type 'DocumentReference' is malformed


The value '2025-09-10T00:00:00' of Property 'date' of Type 'DocumentReference' is a malformed 'instant'. It should match the Regex '([0-9]([0-9]([0-9][1-9]|[1-9]0)|[1-9]00)|[1-9]000)-(0[1-9]|1[0-2])-(0[1-9]|[1-2][0-9]|3[0-1])T([01][0-9]|2[0-3]):[0-5][0-9]:([0-5][0-9]|60)(\.[0-9]+)?(Z|(\+|-)((0[0-9]|1[0-3]):[0-5][0-9]|14:00))'




We can see two problems here - the patient reference is wrong, and the date is malformed because it doesnt have the timezone as a single letter value. Lets fix them and try again: 

In [143]:
note_resource.subject = {"reference": "Patient/3600"}
note_resource.date = datetime(2025, 9, 10).isoformat() + "Z"


In [145]:
res = requests.post(baseURL+"DocumentReference", data=note_resource.model_dump_json(),
                   headers={"Content-Type": "application/fhir+json"}, auth=HTTPBasicAuth(username, password))

print(res)


<Response [201]>


In [151]:
res = requests.get(baseURL+"DocumentReference?patient=3600",
                   headers=get_headers, auth=HTTPBasicAuth(username, password))
print(res)

<Response [200]>


In [163]:
resource = res.json()["entry"][0]["resource"]
# print(resource)
print(bytes.fromhex(resource["content"][0]["attachment"]["data"]).decode("utf-8"))

Patient Name: Clark Kent
DOB: [Redacted]
Date of Visit: 09/10/2025
Presenting Complaint:
Patient presents with complaints of persistent chest discomfort and intermittent episodes of spontaneous x-ray vision activation, particularly during emotionally charged situations (e.g., office meetings, romantic encounters, surprise birthday parties).
History of Present Illness:
Mr. Kent reports a recent increase in involuntary activation of his x-ray vision, resulting in several awkward social encounters, including accidentally seeing through three floors of the Daily Planet building. He also notes mild chest tightness after flying through a cumulonimbus cloud at Mach 3, which he describes as “like being hugged by a thunderstorm.”
Past Medical History:

Kryptonian physiology
Solar overexposure syndrome (resolved)
Mild allergic reaction to kryptonite (ongoing)

Medications:

None (immune to most Earth-based pharmaceuticals)

Allergies:

Kryptonite (severe photosensitivity, weakness, existential d